In [20]:
import pandas as pd 
import numpy as np

In [21]:
# Read data ---------------------------------------------------------------
cig_data = pd.read_csv("/Users/ellenwu/homework3_attempt1-/data/input/The_Tax_Burden_on_Tobacco__1970-2019_20250304.csv")
cpi_data = pd.read_excel("/Users/ellenwu/homework3_attempt1-/data/input/SeriesReport-20250304165118_4f02a8.xlsx", skiprows=11)

/Users/ellenwu/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [22]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("/Users/ellenwu/homework3_attempt1-/data/input/SeriesReport-20250304165118_4f02a8.xlsx", skiprows=11)

# Show the first few rows
print(df.head())

   Year   Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov  \
0  1913   9.8   9.8   9.8   9.8   9.7   9.8   9.9   9.9  10.0  10.0  10.1   
1  1914  10.0   9.9   9.9   9.8   9.9   9.9  10.0  10.2  10.2  10.1  10.2   
2  1915  10.1  10.0   9.9  10.0  10.1  10.1  10.1  10.1  10.1  10.2  10.3   
3  1916  10.4  10.4  10.5  10.6  10.7  10.8  10.8  10.9  11.1  11.3  11.5   
4  1917  11.7  12.0  12.0  12.6  12.8  13.0  12.8  13.0  13.3  13.5  13.5   

    Dec  HALF1  HALF2  
0  10.0    NaN    NaN  
1  10.1    NaN    NaN  
2  10.3    NaN    NaN  
3  11.6    NaN    NaN  
4  13.7    NaN    NaN  


/Users/ellenwu/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [23]:
# Clean tobacco data ------------------------------------------------------
def map_measure(desc):
    mapping = {
        "Average Cost per pack": "cost_per_pack",
        "Cigarette Consumption (Pack Sales Per Capita)": "sales_per_capita",
        "Federal and State tax as a Percentage of Retail Price": "tax_percent",
        "Federal and State Tax per pack": "tax_dollar",
        "Gross Cigarette Tax Revenue": "tax_revenue",
        "State Tax per pack": "tax_state"
    }
    return mapping.get(desc, None)

cig_data['measure'] = cig_data['SubMeasureDesc'].map(map_measure)

cig_data = cig_data[['LocationAbbr', 'LocationDesc', 'Year', 'Data_Value', 'measure']].rename(
    columns={
        'LocationAbbr': 'state_abb',
        'LocationDesc': 'state',
        'Data_Value': 'value'
    }
)

final_data = cig_data.pivot_table(index=['state', 'Year'], 
                                  columns='measure', 
                                  values='value').reset_index()

final_data = final_data.sort_values(by=['state', 'Year'])

In [24]:
# Clean CPI data ----------------------------------------------------------
cpi_data = cpi_data.melt(id_vars=['Year'], 
                         value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                         var_name='month', 
                         value_name='index')

cpi_data = cpi_data.groupby('Year', as_index=False).agg({'index': 'mean'})

In [19]:
# Form final dataset ------------------------------------------------------
# adjust to 2010 dollars (using 2010 CPI = 218)
final_data = final_data.merge(cpi_data, on='Year', how='left')

final_data['price_cpi'] = final_data['cost_per_pack'] * (218 / final_data['index'])

# Write output files ------------------------------------------------------
final_data.to_csv("/Users/ellenwu/homework3_attempt1-/data/output/TaxBurden_Data.txt", sep='\t', index=False)
final_data.to_csv("/Users/ellenwu/homework3_attempt1-/data/output/TaxBurden_Data.csv", index=False)
